
# Lesson XX: Introduction to Geopandas
In this lesson we introduce **geopandas** and some additional spatial python packages.


---


## General information

### Sources

This lesson is inspired by a workshop from a course at [TU Berlin on energy system modelling with data](https://github.com/fneum/data-science-for-esm) written by Fabian Neumann (Copyright (c) 2022, Fabian Neumann). This version was adapted  by Ruth Hamilton.

### About this document

This is a [Google Colab Notebook](https://colab.research.google.com/?utm_source=scs-index). This particular notebook is designed to introduce you to a few of the basic concepts of programming in Python. Like other common notebook formats (e.g. [Jupyter](http://jupyterlab.readthedocs.io/en/stable/) ), the contents of this document are divided into cells, which can contain:

*   Markdown-formatted text,
*   Python code, or
*   raw text

You can execute a snippet of code in a cell by pressing **Shift-Enter** or by pressing the **Run Cell** button that appears when your cursor is on the cell .



---


# Introduction to `geopandas` and `cartopy`

## Basic Setup

We will be using `pandas` and `matplotlib` in this tutorial as we have previously.

Alongside this, we will also be using some additional packages:
- [geopandas](https://geopandas.org/en/stable/index.html) - a package developed to make working with geospatial data in python easier;
- [cartopy](https://scitools.org.uk/cartopy/docs/v0.15/index.html//) - a pacakage designed to make drawing maps for data analysis and visualisation as easy as possible; and



We can install these using the `!pip install...` command we use to install  pandas. Don't forget to import them, too...

In [ ]:
!pip install pandas geopandas matplotlib cartopy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

We'll also supress a few warnings.

In [ ]:
import warnings

warnings.filterwarnings("ignore")

## Why do we need something other than `pandas`?

Let's look at some data. We are going to be using a dataset of conventional power plants in Europe.

Load the dataset  as a pandas DataFrame and look at the first 3 rows.

In [ ]:

fn="https://raw.githubusercontent.com/ruth-ham/TRP479_2023_2024/main/Geopandas/powerplants.csv"
ppl = pd.read_csv(fn, index_col=0)

In [ ]:
ppl.head(3)

This dataset includes coordinates (latitude and longitude), which allows us to  plot the location and capacity of all power plants in a scatter plot. We can use the **pandas** python plotting function to do this ([documentation here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.scatter.html)).

The line `ppl.plot.scatter("lon", "lat", s=ppl.Capacity / 1e3)` uses the latitude and longitude attributes as `x,y` values for the scatter plot and the the Capacity attribute to determine the *size* of each point via the size argument, `s`.

In [ ]:
ppl.plot.scatter("lon", "lat", s=ppl.Capacity / 1e3)



This shows the recognisable shape of Europe, however, the figure lacks the geogprahic context that we'd normally expect for a map; features like shorelines and country borders etc.

To add those, we need to use **Geopandas**.

## Geopandas - a Pandas extension for geospatial data

<img src="https://geopandas.org/en/stable/_images/geopandas_logo.png" width="400px" />

Geopandas extends `pandas` by adding support for geospatial data.

The core data structure in GeoPandas is the `geopandas.GeoDataFrame`, a subclass of `pandas.DataFrame`, that can store geometry columns and perform spatial operations.

<img src="https://geopandas.org/en/stable/_images/dataframe.svg" width="600px" />

:::{note}
Documentation for this package is available at https://geopandas.org/en/stable/.
:::

Typical geometries are points, lines, and polygons. They come from another library called [`shapely`](https://shapely.readthedocs.io/en/stable/manual.html), which helps you create, analyze, and manipulate two-dimensional shapes and their properties, such as points, lines, and polygons.

<img src="https://geobgu.xyz/py/_images/simple_feature_types.svg" width="600px" />

First, we need to import the `geopandas` package (remember, we already installed it using the `!pip install...` command earlier). The conventional alias is `gpd`:

In [ ]:
import geopandas as gpd

The first step is to convert the latitude and longitude attributes given in the dataset to formal spatial geometries. We do this using the `gpd.points_from_xy()` function (this is analagous to similar processes of converting `.csv` data to shapefiles in ArcGIS and QGIS). Once we have converted the `x,y` attributes to **point** geometry, we can convert the pandas DataFrame to a GeoDataFrame using the geopandas `gpd.GeoDataFrame` function. As with any spatial data, we also need to specify a coordinate reference system (CRS). In this case, we are using longitude and latitude values which means that the code '4326' is appropriate.

**Remember** the functions with the `gpd.` prefix are from the **geopandas** package which we *imported* as `gpd`.

In [ ]:
geometry = gpd.points_from_xy(ppl["lon"], ppl["lat"])
gdf_ppl = gpd.GeoDataFrame(ppl, geometry=geometry, crs=4326)

Now, our GeoDataFrame (`gdf_ppl`) looks like this (note the final column):

In [ ]:
gdf_ppl.head(3)

With the additional `geometry` columns, it is now even easier to plot the geographic data (note, we don't even have to specify the geometry attribute):

In [ ]:
gdf_ppl.plot(
    markersize=gdf_ppl.Capacity / 1e2,
)

To symbolise the data based on an attribute, we can simply add the `column` argument.

In [ ]:
gdf_ppl.plot(
    column="Fueltype",
    markersize=gdf_ppl.Capacity / 1e2,
)

We can also start up an interactive map to explore the geodata in more detail. This uses the `explore` function in geopandas to build an interactive map using folium/leaflet.js. You can find out more about the `explore` function [here](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html).





In [ ]:
gdf_ppl.explore(column="Fueltype",width = 800, height= 800)

## Map Projections with Cartopy

<img src="https://scitools.org.uk/cartopy/docs/v0.16/_images/sphx_glr_logo_001.png" width="300px" />

Cartopy is a Python package designed for geospatial data processing and has exposed an interface to enable easy map creation using `matplotlib`.

The Earth is a globe, but we present maps usually on two-dimensional surfaces. Hence, we typically need to *project* data points onto flat surfaces (e.g. screens, paper). However, we will always loose some information in doing so.


<img src="https://raw.githubusercontent.com/SciTools/cartopy-tutorial/master/static/orange_peel.jpg" width="500px" />


A map projection is:

> a systematic transformation of the latitudes and longitudes of locations from the surface of a sphere or an ellipsoid into locations on a plane. [Wikipedia: Map projection](https://en.wikipedia.org/wiki/Map_projection).


Different projections **preserve different metric properties**. As a result,
converting geodata from one projection to another is a common exercise in geographic data science.

- **conformal projections** preserve angles/directions (e.g. Mercator projection)
- **equal-area projections** preserve area measure (e.g. Mollweide)
- **equidistant projections** preserve distances between points (e.g. Plate carrée)
- **compromise projections** seek to strike a balance between distortions (e.g. Robinson)

If you like the "Orange-as-Earth" analogy for projections, checkout [this numberphile video](https://www.youtube.com/watch?v=D3tdW9l1690) by Hannah Fry.

---
**Note**
>Documentation for this package is available at https://scitools.org.uk/cartopy/docs/latest/.
---

First, we need to import the relevant parts of the `cartopy` package:

In [ ]:
import cartopy
import cartopy.crs as ccrs

Let's draw a first map with `cartopy` outlining the global coastlines in the so-called [plate carrée projection (equirectangular projection)](https://en.wikipedia.org/wiki/Equirectangular_projection):

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()


A list of the available projections can be found on the [Cartopy projection list page](https://scitools.org.uk/cartopy/docs/latest/reference/projections.html#cartopy-projections).

In [ ]:
ax = plt.axes(projection=ccrs.Mollweide())
#ax.coastlines()
ax.stock_img()

The methods `.coastlines()` and `.stock_img()` are defined within the `cartopy` package and are intended to integrate with `matplotlib`. You can find a list of the methods in the *'Cartopy matplotlib integration reference document'* [here](https://scitools.org.uk/cartopy/docs/v0.15/matplotlib/geoaxes.html).

We can combine the functionality of `cartopy` with `geopandas` plots:

In [ ]:
fig = plt.figure(figsize=(7, 7)) #creates a matplotlib figure instance

ax = plt.axes(projection=ccrs.PlateCarree()) #adds axes to the figure and specifies a projection

gdf_ppl.plot(
    ax=ax,
    column="Fueltype",
    markersize=gdf_ppl.Capacity / 1e2,
)  #adds the gdf_ppl data to the axes

We can add further [geographic features](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html#the-cartopy-feature-interface) to this map for better orientation.

For instance, we can add the coastlines...

In [ ]:
ax.coastlines()
fig

... country borders ...

In [ ]:
ax.add_feature(cartopy.feature.BORDERS, color="grey", linewidth=0.5)
fig

... colour in the ocean in blue ...

In [ ]:
ax.add_feature(cartopy.feature.OCEAN, color="azure")
fig

...and color in the land area in yellow ...

In [ ]:
ax.add_feature(cartopy.feature.LAND, color="cornsilk")
fig

Geopandas will automatically calculate sensible bounds for the plot given the geographic data.
But we can also manually zoom in or out by setting the spatial extent with the `.set_extent()` method:

In [ ]:
ax.set_extent([3, -9, 49, 61])
fig

### Reprojecting a `GeoDataFrame`

In `geopandas`, we can use the function `.to_crs()` to convert a `GeoDataFrame` to a desired coordinate reference system. In this particular case, we use the `proj4_init` string of an initialised `cartopy` projection to reproject our power plant `GeoDataFrame`.

> A `proj4_init` string is a text-based representation of a coordinate reference system (CRS) that defines the parameters for transforming geographical coordinates between different spatial reference systems, used by the PROJ library. It will look similar to this: "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs".

You can find the list of Cartopy projections [here](https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html#osgb).

In [ ]:

fig = plt.figure(figsize=(7, 7))

#specify new_crs variable to desired projection (British National Grid)
new_crs = ccrs.OSGB()

#create axes object with using new projection
ax = plt.axes(projection=new_crs)

#reproject geodataframe (gdf) to new crs (new_crs) using .to_crs function.
gdf_ppl.to_crs(new_crs.proj4_init).plot(
    ax=ax,
    column="Fueltype",
    markersize=gdf_ppl.Capacity / 1e2,
)


ax.coastlines()


## Reading and Writing Files with `geopandas`

In the following example, we'll load a dataset containing the [NUTS regions](https://en.wikipedia.org/wiki/Nomenclature_of_Territorial_Units_for_Statistics):

> Nomenclature of Territorial Units for Statistics or NUTS (French: Nomenclature des unités territoriales statistiques) is a geocode standard for referencing the subdivisions of countries for statistical purposes.

Our ultimate **goal** for this part of the tutorial is to map the power plant capacities to the NUTS-2 region they belong to.

<img src="https://ec.europa.eu/eurostat/documents/345175/501899/framing" width="400px" />

Common filetypes for vector-based geospatial datasets are GeoPackage (`.gpkg`), GeoJSON (`.geojson`), File Geodatabase (`.gdb`), or Shapefiles (`.shp`).

In `geopandas` we can use the `gpd.read_file()` function to read such files. So let's start:

In [ ]:
url="https://raw.githubusercontent.com/ruth-ham/TRP479_2023_2024/main/Geopandas/NUTS_RG_10M_2021_4326.geojson"

In [ ]:
nuts = gpd.read_file(url)

In [ ]:
nuts.head(3)

It is good practice to set an index. This is because looking up rows by index is much faster than looking up rows by column value.

You can use `.set_index()` to do that:

In [ ]:
nuts = nuts.set_index("id")

In [ ]:
nuts.head(3)

We can also check out the geometries in the dataset with `.geometry`:

In [ ]:
nuts.geometry

With `.crs` we can check in which coordinate reference system the data is given:

In [ ]:
nuts.crs

In [ ]:
nuts.total_bounds




Let's filter by NUTS-2 level...

In [ ]:

nuts2 = nuts.query("LEVL_CODE == 2")

... and explore what kind of geometries we have in the dataset ...

In [ ]:
nuts2.explore(width=600, height=600)

To write a GeoDataFrame back to file use `GeoDataFrame.to_file()`. The file format is inferred from the file ending.

In [ ]:
nuts2.to_file("tmp.geojson")

## Calculating the areas and buffers

The first thing we need to do to calculate area or buffers is to reproject the `GeoDataFrame` to an equal-area projection (here: [EPSG:3035](https://epsg.io/3035) which is valid only within Europe; global alternative is the Mollweide projection [EPSG:54009](https://epsg.io/54009)):

In [ ]:
nuts2 = nuts2.to_crs(3035)

The area can be accessed via `.area` and is given in m² (after projection). Let's convert to km²:

In [ ]:
area = nuts2.area / 1e6
area

In [ ]:
nuts2.explore(column=area, vmax=1e5,width=600, height=600)

We can also build a buffer of 1km around each geometry using `.buffer()`:

In [ ]:
nuts2.buffer(1000).explore(width=600, height=600)

## Joining spatial datasets

Multiple `GeoDataFrames` can be combined via *spatial joins*.

Observations from two datasets are combined with the [`.sjoin()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html) function based on their **spatial relationship** to one another (e.g. whether they are intersecting or overlapping). You can read more about the specific options [here](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#binary-predicate-joins).

To perform a spatial join, you need to provide the following information:

- The two GeoDataFrames you want to join (e.g., `left_df` and `right_df`).
- The type of spatial relationship to test (e.g., `intersects`, `contains`, `within`). This is specified using the `op` parameter.
- The type of join to perform (e.g., `inner`, `left`, `right`). This is specified using the `how` parameter.

The `.sjoin()` function will then iterate through the geometries in both GeoDataFrames, evaluate the specified spatial relationship, and join the matching records.

For example, if the `op` parameter is set to 'intersects', the function will check if the geometries of each record in `left_df` intersect with the geometries of any records in `right_df`. If a match is found, the attributes of the corresponding records from both GeoDataFrames will be combined into a new record in the output GeoDataFrame.

By performing spatial joins, you can efficiently combine and analyze geospatial data based on their spatial relationships, without the need for explicit coordinate-based calculations.

Let's first reproject the `gdf_ppl` object to the same CRS as `nuts2`:

In [ ]:
gdf_ppl = gdf_ppl.to_crs(3035)

Then, let's have a look at both datasets at once. We want to find out which points (representing power plants) lie within which shape (representing NUTS regions).

In [ ]:
fig = plt.figure(figsize=(7, 7))

ax = plt.axes(projection=ccrs.epsg(3035))

nuts2.plot(ax=ax, edgecolor="black", facecolor="lightgrey")

gdf_ppl.to_crs(3035).plot(
    ax=ax, column="Fueltype", markersize=gdf_ppl.Capacity / 20, legend=True
)

ax.set_extent([3, -9, 49, 61])

We can now apply the `.sjoin` function to look for which power plants lie within which NUTS1 region. By default, `sjoin` looks for intersections and keeps the geometries of the *left* `GeoDataFrame`.

In [ ]:
joined = gdf_ppl.sjoin(nuts2)

If we look at this new `GeoDataFrame`, we now have additional columns from the NUTS1 data:

In [ ]:
joined.head(3)

We can now use these new columns to group the capacities (and convert to a suitable unit):

In [ ]:
cap = joined.groupby("NUTS_ID").Capacity.sum() / 1000  # GW

The variable `cap` should now contain the *total capacity* (in GigaWatts) of each NUTS2 region.

In [ ]:
cap

Let's quickly check if all NUTS2 regions have power plants:

In [ ]:
nuts2.index.difference(cap.index)

This is not the case. Then it is good practice to reindex the series to include all NUTS2 regions, even if this leads to some NaN values.

In [ ]:
cap = cap.reindex(nuts2.index)

In [ ]:
cap

Finally, we can plot the total generation capacity per NUTS1 region on a map.

In [ ]:
nuts2.head(3)

In [ ]:
nuts2.plot(figsize=(7, 7), column=cap, legend=True)

In [ ]:
fig = plt.figure(figsize=(7, 7))

ax = plt.axes(projection=ccrs.epsg(3035))
plt.set_cmap("Oranges") #changes colormap

nuts2.plot(ax=ax,column=cap, edgecolor="black", facecolor="lightgrey",legend=True,linewidth=0.2)


ax.set_extent([3, -9, 49, 61]) # set extent to UK - note that dataset is europewide
plt.title("Power generation in GW by NUTS-2 region") #adds title

This concludes the `geopandas` and `cartopy` tutorial.

## Exercises

**Task 1:** Recreate the figure above (i.e. generation capacity per NUTS2 region)
- using 3 different [cartopy projections](https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html) of your choice.
- such that the capacities are normalised by the **area** of the NUTS2 region (unit: MW/km²).
- such that it only shows the **hard coal power** plant capacities.
- such that it only shows NUTS2 shapes for Germany.
- with NUTS3 regions instead of NUTS2 regions.

**Question:** Which UK NUTS2 regions have the highest conventional generation capacity? Can you think of reasons why?

---
#Answers

---
Once you have had a go at these tasks, you can check your approach against the code boxes below.
---

In [ ]:
# @title Code to show using 3 different cartopy projections
for crs in [ccrs.EqualEarth(), ccrs.AlbersEqualArea(), ccrs.Orthographic()]:
    fig = plt.figure()
    ax = plt.axes(projection=crs)
    nuts2.to_crs(crs.proj4_init).plot(ax=ax, column=cap, legend=True)

In [ ]:
# @title Normalised by area in MW/km²:
nuts2.plot(figsize=(7, 7), column=cap / (nuts2.area / 1e9), legend=True)

In [ ]:
# @title Only hard coal generators:
hard_coal_cap = (
    joined.query("Fueltype == 'Hard Coal'")
    .groupby("NUTS_ID")
    .Capacity.sum()
    .reindex(nuts2.index)
    .div(1e3)
)
nuts2.plot(
    figsize=(7, 7),
    column=hard_coal_cap,
    legend=True,
    missing_kwds=dict(color="lightgrey"),
)
plt.ylim(1e6, 6e6)
plt.xlim(2e6, 6e6);

In [ ]:
# @title Code to show only Germany
subregion = nuts2.query("CNTR_CODE == 'DE'")
subregion.plot(column=cap.reindex(subregion.index), legend=True)

In [ ]:
# @title Code to show only Germany and Denmark:
countries = ["DE", "DK"]
#subregion = nuts2.query("CNTR_CODE in @countries")  # alternative A
subregion = nuts2.loc[nuts2.CNTR_CODE.isin(countries)]  # alternative B
ax = plt.axes(projection=ccrs.epsg(3035))
subregion.plot(ax=ax,column=cap.reindex(subregion.index), legend=True)

ax.add_feature(cartopy.feature.BORDERS, color="black", linewidth=0.5)
ax.coastlines(linewidth=0.75,color="grey")

In [ ]:
# @title Show NUTS3 data
nuts3 = nuts.query("LEVL_CODE == 3").to_crs(3035)
joined3 = gdf_ppl.sjoin(nuts3)
cap3 = joined3.groupby("NUTS_ID").Capacity.sum().reindex(nuts3.index).div(1000)  # GW
nuts3.plot(figsize=(7, 7), column=cap3, legend=True)

In [ ]:
# @title Find NUTS2 region in UK with highest capacity:
#countries=["UK"]
nuts_uk = nuts2.loc[nuts2.CNTR_CODE.isin(["UK"])]  # alternative B
joined_uk = gdf_ppl.sjoin(nuts_uk)
cap_uk = joined_uk.groupby("NUTS_ID").Capacity.sum().reindex(nuts_uk.index).div(1000)  # GW

cap_uk.sort_values(ascending=False).head(5)  # alternative A

In [ ]:
# @title Identify top 5 NUTS2 UK regions by capacity
#find top 5 UK NUTS2 region codes by capacity
top_5=cap_uk.sort_values(ascending=False).head(5)

#find information about NUTS2 regions from NUTS data
nuts[nuts['NUTS_ID'].isin(list(top_5.index.values))] #NOTE these are NOT returned in capacity order!
